In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import sqlite3 as sl

conn = sl.connect('my-test.db')


In [15]:
with conn:
    conn.execute("""
        CREATE TABLE test_table (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER
        );
    """)

OperationalError: table test_table already exists

In [4]:
sql = 'INSERT INTO test_table (id, name, age) values(?, ?, ?)'
data = [
    (1, 'Alice', 21),
    (2, 'Bob', 22),
    (3, 'Chris', 23)
]
with conn:
    conn.executemany(sql, data)


In [20]:
sql = 'INSERT INTO test_table (name, age) values(?, ?)'
data = [
    ('Alice2', 21),
    ('Bob2', 22),
    ('Chris2', 23)
]
with conn:
    conn.executemany(sql, data)


In [29]:
with con:
    data = conn.execute("SELECT * FROM test_table")
    for row in data:
        print(row)

(1, 'Alice', 21)
(2, 'Bob', 22)
(3, 'Chris', 23)
(4, 'Alice2', 21)
(5, 'Bob2', 22)
(6, 'Chris2', 23)
(7, 'Alice2', 21)
(8, 'Alice3', 222221)


In [22]:
c = conn.cursor()

c.execute('SELECT * FROM test_table')
data = c.fetchall()
print(data)



[(1, 'Alice', 21), (2, 'Bob', 22), (3, 'Chris', 23), (4, 'Alice2', 21), (5, 'Bob2', 22), (6, 'Chris2', 23)]


In [28]:
sql = 'INSERT INTO test_table (name, age) values(?, ?)'
data = ('Alice3', 222221)
with conn:
    conn.execute(sql, data)


In [3]:
>>> import structlog
>>> log = structlog.get_logger()
>>> log.msg("greeted", whom="world", more_than_a_string=[1, 2, 3])  


2020-08-26 00:54.50 greeted                        more_than_a_string=[1, 2, 3] whom=world


In [4]:
from simplegmail import Gmail
from simplegmail.query import construct_query

g = Gmail()

query_params = {
    "sender": "no-reply@emails.monoprice.com",
    # "sender": "notification@facebookmail.com",
}

messages = g.get_messages(query=construct_query(query_params))


Fetched 30 emails in 1 batches thus far.

In [5]:
len(messages)

30

In [16]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine("sqlite:///my-test.db", echo=True)

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

    def __repr__(self):
        return f'User {self.name}'




In [17]:
Base.metadata.create_all(engine)


2020-08-26 03:08:55,452 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-26 03:08:55,453 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 03:08:55,454 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-26 03:08:55,455 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 03:08:55,456 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2020-08-26 03:08:55,456 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 03:08:55,457 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2020-08-26 03:08:55,458 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 03:08:55,459 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2020-08-26 03:08:55,460 INFO sqlalchemy.engine.base.Engine ()
2020-08-26 03:08:55,462 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
user = User(name='John Snow', password='johnspassword')
session.add(user)

print(user.id)  # None


None


In [19]:
session.commit()

2020-08-26 03:10:13,914 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-26 03:10:13,916 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, password) VALUES (?, ?)
2020-08-26 03:10:13,917 INFO sqlalchemy.engine.base.Engine ('John Snow', 'johnspassword')
2020-08-26 03:10:13,919 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
user.id

1

In [ ]:
time_created = Column(DateTime(timezone=True), server_default=func.now())


In [9]:
vars(messages[0])

{'_service': <googleapiclient.discovery.Resource at 0x10c121b10>,
 'user_id': 'me',
 'id': '1742723cd673df26',
 'thread_id': '1742723cd673df26',
 'recipient': 'kuanche@gmail.com',
 'sender': 'Monoprice <no-reply@emails.monoprice.com>',
 'subject': 'Deal Alert >> 4 for $20 + Free Shipping Luxe Series MFi Certified Lightning Cable (Black,3ft)',
 'date': '2020-08-25 15:38:02-04:00',
 'snippet': "Hurry, While Supplies Last! \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c \u200c Can't see this email? View it in your Browser. Monoprice logo New Products Monoprice Finds Wall Mount Finder Cable Finder Clearance",
 'plain': "Hurry, While Supplies Last!\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Can't see this email? View it in your Browser.(http://enews.emails.monoprice.com/q/l3SA0TJNXwqo0gpIPA6qDU0Rrb2aAu7tF5NgFCWrMEVWr1EQ2_3mMA6H3)\r\n\r\nMonoprice logo (http://enews.emails.monoprice.com/q/cm3Er

In [39]:
from time import sleep

for i in range(100): 
    sleep(1)
    print(f"\rThis will be printed on the same line {i}", end="")



This will be printed on the same line 99

In [7]:
import pandas as pd 

raw_df = pd.DataFrame([vars(f) for f in messages])

df = raw_df[['id', 'recipient', 'sender', 'subject', 'date', 'snippet', 'label_ids']]

df = df.set_index('id')

df



,recipient,sender,subject,date,snippet,label_ids
id,,,,,,
17312d2adc16eea0,Kuan Cheng <kuanche@gmail.com>,Janet Li <notification@facebookmail.com>,Janet Li sent you a message.,2020-07-02 23:57:39-04:00,Janet Li sent you a message. Open Messenger Yo...,"[UNREAD, CATEGORY_SOCIAL, INBOX]"
17312cad23274973,Kuan Cheng <kuanche@gmail.com>,Steven Lai <notification@facebookmail.com>,Steven Lai sent a message to your group conver...,2020-07-02 23:48:50-04:00,Steven Lai sent a message to your group conver...,"[UNREAD, CATEGORY_SOCIAL, INBOX]"
173128e33f0644d5,Kuan Cheng <kuanche@gmail.com>,Isabelle Ahn Richardson <notification@facebook...,Isabelle Ahn Richardson sent a message to your...,2020-07-02 22:42:39-04:00,Isabelle Ahn Richardson sent a message to your...,"[UNREAD, CATEGORY_SOCIAL, INBOX]"
17312101f5cadccc,Kuan Cheng <kuanche@gmail.com>,Isabelle Ahn Richardson <notification@facebook...,Isabelle Ahn Richardson sent a message to your...,2020-07-02 20:24:58-04:00,Isabelle Ahn Richardson sent a message to your...,"[UNREAD, CATEGORY_SOCIAL, INBOX]"
1730d25b6a222fe9,Kuan Cheng <kuanche@gmail.com>,Isabelle Ahn Richardson <notification@facebook...,Isabelle Ahn Richardson sent a message to your...,2020-07-01 21:30:31-04:00,Isabelle Ahn Richardson sent a message to your...,"[UNREAD, CATEGORY_SOCIAL, INBOX]"
...,...,...,...,...,...,...
16f61d0ec973fd54,Kuan Cheng <kuanche@gmail.com>,Peter Tia <notification@facebookmail.com>,"Peter Tia sent a message to the conversation ""...",2020-01-01 10:54:39-05:00,"Peter Tia sent a message to the conversation ""...","[UNREAD, CATEGORY_SOCIAL, INBOX]"
16f61a3fde7d0f00,Kuan Cheng <kuanche@gmail.com>,Janet Li <notification@facebookmail.com>,Janet Li sent you a message.,2020-01-01 10:05:25-05:00,Janet Li sent you a message. Open Messenger Yo...,"[UNREAD, CATEGORY_SOCIAL, INBOX]"
16f6144fe11199d0,Kuan Cheng <kuanche@gmail.com>,Bohan Liu <notification@facebookmail.com>,"Bohan Liu sent a message to the conversation ""...",2020-01-01 08:21:45-05:00,"Bohan Liu sent a message to the conversation ""...","[UNREAD, CATEGORY_SOCIAL, INBOX]"


In [20]:
import pandas as pd

class Foo():
    def __init__(self, a):
        self.a = a
    def __repr__(self):
        return "Foo({})".format(self.a)

df = pd.DataFrame([[Foo(1), 1], [Foo(1), 2], [Foo(2), 3]])

df

g = df.groupby(df[0].apply(lambda x: x.a))  # Note: x is of type Foo

g.filter(lambda x: x.name == 1)

,0,1
0,Foo(1),1
1,Foo(1),2


In [21]:
df

,0,1
0,Foo(1),1
1,Foo(1),2
2,Foo(2),3
